<a href="https://colab.research.google.com/github/Laxmanlee/Anime-Recoomendation-system/blob/main/Copy_of_BD_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import seaborn as sns
%matplotlib inline

In [3]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [4]:
anime = pd.read_csv("anime.csv")

In [5]:
anime.head()

,anime_id,name,genre,type,episodes,rating,members
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262
3,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,673572
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266


In [6]:
anime.isnull().sum()

anime_id      0
name          0
genre        62
type         25
episodes      0
rating      230
members       0
dtype: int64

In [7]:
anime[anime['episodes']=='Unknown'].head(3)

,anime_id,name,genre,type,episodes,rating,members
74,21,One Piece,"Action, Adventure, Comedy, Drama, Fantasy, Sho...",TV,Unknown,8.58,504862
252,235,Detective Conan,"Adventure, Comedy, Mystery, Police, Shounen",TV,Unknown,8.25,114702
615,1735,Naruto: Shippuuden,"Action, Comedy, Martial Arts, Shounen, Super P...",TV,Unknown,7.94,533578


In [8]:
anime.loc[(anime["genre"]=="Hentai") & (anime["episodes"]=="Unknown"),"episodes"] = "1"
anime.loc[(anime["type"]=="OVA") & (anime["episodes"]=="Unknown"),"episodes"] = "1"

anime.loc[(anime["type"] == "Movie") & (anime["episodes"] == "Unknown")] = "1"

In [9]:
known_animes = {"Naruto Shippuuden":500, "One Piece":784,"Detective Conan":854, "Dragon Ball Super":86,
                "Crayon Shin chan":942, "Yu Gi Oh Arc V":148,"Shingeki no Kyojin Season 2":25,
                "Boku no Hero Academia 2nd Season":25,"Little Witch Academia TV":25}

In [10]:
for k,v in known_animes.items():    
    anime.loc[anime["name"]==k,"episodes"] = v

In [11]:
anime["episodes"] = anime["episodes"].map(lambda x:np.nan if x=="Unknown" else x)

In [12]:
anime["episodes"].fillna(anime["episodes"].median(),inplace = True)

In [13]:
pd.get_dummies(anime[["type"]]).head()

,type_1,type_Movie,type_Music,type_ONA,type_OVA,type_Special,type_TV
0,0,1,0,0,0,0,0
1,0,0,0,0,0,0,1
2,0,0,0,0,0,0,1
3,0,0,0,0,0,0,1
4,0,0,0,0,0,0,1


In [14]:
anime["rating"] = anime["rating"].astype(float)
anime["rating"].fillna(anime["rating"].median(),inplace = True)
anime["members"] = anime["members"].astype(float)

In [15]:
anime_features = pd.concat([anime["genre"].str.get_dummies(sep=","),
                            pd.get_dummies(anime[["type"]]),
                            anime[["rating"]],anime[["members"]],anime["episodes"]],axis=1)
anime["name"] = anime["name"].map(lambda name:re.sub('[^A-Za-z0-9]+', " ", name))
anime_features.head()

,Adventure,Cars,Comedy,Dementia,Demons,Drama,Ecchi,Fantasy,Game,Harem,...,type_1,type_Movie,type_Music,type_ONA,type_OVA,type_Special,type_TV,rating,members,episodes
0,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,9.37,200630.0,1
1,1,0,0,0,0,1,0,1,0,0,...,0,0,0,0,0,0,1,9.26,793665.0,64
2,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,9.25,114262.0,51
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,9.17,673572.0,24
4,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,9.16,151266.0,51


In [16]:
anime_features.columns

Index([' Adventure', ' Cars', ' Comedy', ' Dementia', ' Demons', ' Drama',
       ' Ecchi', ' Fantasy', ' Game', ' Harem', ' Hentai', ' Historical',
       ' Horror', ' Josei', ' Kids', ' Magic', ' Martial Arts', ' Mecha',
       ' Military', ' Music', ' Mystery', ' Parody', ' Police',
       ' Psychological', ' Romance', ' Samurai', ' School', ' Sci-Fi',
       ' Seinen', ' Shoujo', ' Shoujo Ai', ' Shounen', ' Shounen Ai',
       ' Slice of Life', ' Space', ' Sports', ' Super Power', ' Supernatural',
       ' Thriller', ' Vampire', ' Yaoi', ' Yuri', '1', 'Action', 'Adventure',
       'Cars', 'Comedy', 'Dementia', 'Demons', 'Drama', 'Ecchi', 'Fantasy',
       'Game', 'Harem', 'Hentai', 'Historical', 'Horror', 'Josei', 'Kids',
       'Magic', 'Martial Arts', 'Mecha', 'Military', 'Music', 'Mystery',
       'Parody', 'Police', 'Psychological', 'Romance', 'Samurai', 'School',
       'Sci-Fi', 'Seinen', 'Shoujo', 'Shounen', 'Slice of Life', 'Space',
       'Sports', 'Super Power', 'Supernat

In [17]:
from sklearn.preprocessing import MinMaxScaler

In [18]:
min_max_scaler = MinMaxScaler()
anime_features = min_max_scaler.fit_transform(anime_features)

In [19]:
np.round(anime_features,2)

array([[0.  , 0.  , 0.  , ..., 0.93, 0.2 , 0.  ],
       [1.  , 0.  , 0.  , ..., 0.92, 0.78, 0.03],
       [0.  , 0.  , 1.  , ..., 0.92, 0.11, 0.03],
       ...,
       [0.  , 0.  , 0.  , ..., 0.43, 0.  , 0.  ],
       [0.  , 0.  , 0.  , ..., 0.44, 0.  , 0.  ],
       [0.  , 0.  , 0.  , ..., 0.5 , 0.  , 0.  ]])

In [20]:
from sklearn.neighbors import NearestNeighbors

In [21]:
nbrs = NearestNeighbors(n_neighbors=6, algorithm='ball_tree').fit(anime_features)

In [22]:
distances, indices = nbrs.kneighbors(anime_features)

In [23]:
def get_index_from_name(name):
    return anime[anime["name"]==name].index.tolist()[0]

In [24]:
all_anime_names = list(anime.name.values)

In [25]:
def get_id_from_partial_name(partial):
    for name in all_anime_names:
        if partial in name:
            print(name,all_anime_names.index(name))

In [26]:
def print_similar_animes(query=None,id=None):
    if id:
        for id in indices[id][1:]:
            print(anime.df.loc[id]["name"])
    if query:
        found_id = get_index_from_name(query)
        for id in indices[found_id][1:]:
            print(anime.df.loc[id]["name"])

In [27]:
get_id_from_partial_name("Naruto")

Boruto Naruto the Movie 486
Naruto Shippuuden 615
The Last Naruto the Movie 719
Naruto Shippuuden Movie 6 Road to Ninja 784
Naruto 841
Boruto Naruto the Movie Naruto ga Hokage ni Natta Hi 1103
Naruto Shippuuden Movie 5 Blood Prison 1237
Naruto x UT 1343
Naruto Shippuuden Movie 4 The Lost Tower 1472
Naruto Shippuuden Movie 3 Hi no Ishi wo Tsugu Mono 1573
Naruto Shippuuden Movie 1 1827
Naruto Shippuuden Movie 2 Kizuna 1828
Naruto Shippuuden Shippuu quot Konoha Gakuen quot Den 2374
Naruto Honoo no Chuunin Shiken Naruto vs Konohamaru  2416
Naruto SD Rock Lee no Seishun Full Power Ninden 2457
Naruto Shippuuden Sunny Side Battle 2458
Naruto Movie 1 Dai Katsugeki Yuki Hime Shinobu Houjou Dattebayo  2756
Naruto Soyokazeden Movie Naruto to Mashin to Mitsu no Onegai Dattebayo  2997
Naruto Movie 2 Dai Gekitotsu Maboroshi no Chiteiiseki Dattebayo  3449
Naruto Dai Katsugeki Yuki Hime Shinobu Houjou Dattebayo Special Konoha Annual Sports Festival 3529
Naruto Movie 3 Dai Koufun Mikazuki Jima no Anima

In [28]:
anime[['name','rating','members','genre','anime_id','type']].sort_values(by='rating', ascending=False)[:10]

,name,rating,members,genre,anime_id,type
10464,Taka no Tsume 8 Yoshida kun no X Files,10.00,13.0,"Comedy, Parody",33662,Movie
10400,Spoon hime no Swing Kitchen,9.60,47.0,"Adventure, Kids",30120,TV
9595,Mogura no Motoro,9.50,62.0,Slice of Life,23005,Movie
0,Kimi no Na wa,9.37,200630.0,"Drama, Romance, School, Supernatural",32281,Movie
9078,Kahei no Umi,9.33,44.0,Historical,33607,Movie
1,Fullmetal Alchemist Brotherhood,9.26,793665.0,"Action, Adventure, Drama, Fantasy, Magic, Mili...",5114,TV
10786,Yakusoku Africa Mizu to Midori,9.25,53.0,"Drama, Kids",26313,OVA
2,Gintama,9.25,114262.0,"Action, Comedy, Historical, Parody, Samurai, S...",28977,TV
3,Steins Gate,9.17,673572.0,"Sci-Fi, Thriller",9253,TV
4,Gintama 039,9.16,151266.0,"Action, Comedy, Historical, Parody, Samurai, S...",9969,TV


In [29]:
anime[['name','rating','members','genre','anime_id','type']].sort_values(by='members', ascending=False)[:10]

,name,rating,members,genre,anime_id,type
40,Death Note,8.71,1013917.0,"Mystery, Police, Psychological, Supernatural, ...",1535,TV
86,Shingeki no Kyojin,8.54,896229.0,"Action, Drama, Fantasy, Shounen, Super Power",16498,TV
804,Sword Art Online,7.83,893100.0,"Action, Adventure, Fantasy, Game, Romance",11757,TV
1,Fullmetal Alchemist Brotherhood,9.26,793665.0,"Action, Adventure, Drama, Fantasy, Magic, Mili...",5114,TV
159,Angel Beats,8.39,717796.0,"Action, Comedy, Drama, School, Supernatural",6547,TV
19,Code Geass Hangyaku no Lelouch,8.83,715151.0,"Action, Mecha, Military, School, Sci-Fi, Super...",1575,TV
841,Naruto,7.81,683297.0,"Action, Comedy, Martial Arts, Shounen, Super P...",20,TV
3,Steins Gate,9.17,673572.0,"Sci-Fi, Thriller",9253,TV
445,Mirai Nikki TV,8.07,657190.0,"Action, Mystery, Psychological, Shounen, Super...",10620,TV
131,Toradora,8.45,633817.0,"Comedy, Romance, School, Slice of Life",4224,TV


Using rating.csv file


In [30]:
pd.read_csv('anime.csv', usecols= ['name','rating'])

,name,rating
0,Kimi no Na wa.,9.37
1,Fullmetal Alchemist: Brotherhood,9.26
2,Gintama°,9.25
3,Steins;Gate,9.17
4,Gintama&#039;,9.16
...,...,...
12289,Toushindai My Lover: Minami tai Mecha-Minami,4.15
12290,Under World,4.28
12291,Violence Gekiga David no Hoshi,4.88
12292,Violence Gekiga Shin David no Hoshi: Inma Dens...,4.98


In [31]:
user = pd.read_csv('rating.csv')
user.head()

,user_id,anime_id,rating
0,1,20,-1
1,1,24,-1
2,1,79,-1
3,1,226,-1
4,1,241,-1


In [32]:
user = user.rename({'rating':'userRating'}, axis='columns')
user.head()

,user_id,anime_id,userRating
0,1,20,-1
1,1,24,-1
2,1,79,-1
3,1,226,-1
4,1,241,-1


In [33]:
mergedata = pd.merge(anime, user, on=['anime_id','anime_id'])
mergedata = mergedata[mergedata.user_id <= 20000]
mergedata.head()

,anime_id,name,genre,type,episodes,rating,members,user_id,userRating
0,32281,Kimi no Na wa,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630.0,99,5
1,32281,Kimi no Na wa,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630.0,152,10
2,32281,Kimi no Na wa,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630.0,244,10
3,32281,Kimi no Na wa,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630.0,271,10
4,32281,Kimi no Na wa,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630.0,278,-1


In [34]:
mergedata

,anime_id,name,genre,type,episodes,rating,members,user_id,userRating
0,32281,Kimi no Na wa,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630.0,99,5
1,32281,Kimi no Na wa,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630.0,152,10
2,32281,Kimi no Na wa,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630.0,244,10
3,32281,Kimi no Na wa,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630.0,271,10
4,32281,Kimi no Na wa,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630.0,278,-1
...,...,...,...,...,...,...,...,...,...
7813622,5569,Tsui no Sora,Hentai,OVA,1,2.98,934.0,5204,1
7813623,5569,Tsui no Sora,Hentai,OVA,1,2.98,934.0,12233,1
7813654,17823,Ikenai Boy,"Comedy, Harem, Hentai",OVA,1,3.56,179.0,5831,-1
7813676,18197,Nudl Nude,"Comedy, Hentai, Parody",OVA,1,4.14,186.0,14697,-1


In [56]:
user_anime = pd.crosstab(mergedata['user_id'], mergedata['name'])
user_anime.head()

name,,Summer,Summer Specials,hack G U Returner,hack G U Trilogy,hack G U Trilogy Parody Mode,hack Gift,hack Intermezzo,hack Liminality,hack Quantum,...,iDOLM STER Xenoglossia Specials,lilac bombs Jun Togawa,makemagic,s CRY ed,vivi,xxxHOLiC,xxxHOLiC Kei,xxxHOLiC Movie Manatsu no Yoru no Yume,xxxHOLiC Rou,xxxHOLiC Shunmuki
user_id,,,,,,,,,,,,,,,,,,,,,
1,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


Principal component Analysis
Dimensionality Reduction using PCA

In [36]:
from sklearn.decomposition import PCA

pca = PCA(n_components=3, whiten=True)
# this will reduce number of dimensions to 3 which we can easily plot
pca.fit(user_anime)
pca_samples = pca.transform(user_anime)

ps = pd.DataFrame(pca_samples)
ps.head()

,0,1,2
0,0.851932,-1.664197,2.493720
1,-0.839985,-0.268672,0.315131
2,0.059478,-0.840958,-1.513915
3,-0.269433,-0.191942,0.246589
4,2.921967,-1.091926,1.836814


In [37]:
tocluster = pd.DataFrame(ps[[0,1,2]])
tocluster.head()
# 2000x3 - 2000 users and 3 features

,0,1,2
0,0.851932,-1.664197,2.493720
1,-0.839985,-0.268672,0.315131
2,0.059478,-0.840958,-1.513915
3,-0.269433,-0.191942,0.246589
4,2.921967,-1.091926,1.836814


In [38]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

scores = []
inertia_list = np.empty(5)

for i in range(2, 5):
    # we will check cluster in range 2 to 5
    kmeans = KMeans(n_clusters=i)
    kmeans.fit(tocluster)
    inertia_list[i] = kmeans.inertia_
    print(inertia_list[i])
    scores.append(silhouette_score(tocluster, kmeans.labels_))
    print(scores)

46022.15437874266
[0.5090955850182507]
35247.5799825993
[0.5090955850182507, 0.45278764164055485]
26627.530061347457
[0.5090955850182507, 0.45278764164055485, 0.47312934901851467]


K-means Clustering 


In [39]:
from sklearn.cluster import KMeans

clusterer = KMeans(n_clusters=4,random_state=30).fit(tocluster)
centers = clusterer.cluster_centers_
c_preds = clusterer.predict(tocluster)

In [40]:
user_anime['cluster'] = c_preds

user_anime.head(10)

name,,Summer,Summer Specials,hack G U Returner,hack G U Trilogy,hack G U Trilogy Parody Mode,hack Gift,hack Intermezzo,hack Liminality,hack Quantum,...,lilac bombs Jun Togawa,makemagic,s CRY ed,vivi,xxxHOLiC,xxxHOLiC Kei,xxxHOLiC Movie Manatsu no Yoru no Yume,xxxHOLiC Rou,xxxHOLiC Shunmuki,cluster
user_id,,,,,,,,,,,,,,,,,,,,,
1,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3


In [41]:
# here we calculate mean score for all movies in one cluster
# example c0 contains all movies in cluster 0 and their corresponding scores calculated by taking mean from all users
c0 = user_anime[user_anime['cluster']==0].drop('cluster',axis=1).mean()
c1 = user_anime[user_anime['cluster']==1].drop('cluster',axis=1).mean()
c2 = user_anime[user_anime['cluster']==2].drop('cluster',axis=1).mean()
c3 = user_anime[user_anime['cluster']==3].drop('cluster',axis=1).mean()

In [42]:
c0.sort_values(ascending=False)[0:10]

name
Sword Art Online                0.921801
High School DxD                 0.862559
Boku wa Tomodachi ga Sukunai    0.837085
Date A Live                     0.835308
Angel Beats                     0.824052
No Game No Life                 0.800948
Highschool of the Dead          0.779621
Kore wa Zombie Desu ka          0.774882
Shingeki no Kyojin              0.773697
Hataraku Maou sama              0.770735
dtype: float64

In [43]:
columns =  ['id', 'name', '0']

df0 = pd.DataFrame(c0)
df0 = pd.merge(anime, df0, on=['name','name'])
df0 = df0.drop(["genre", "type", "episodes", "rating", "members"], axis=1)
df0.columns = columns
df0

,id,name,0
0,32281,Kimi no Na wa,0.079384
1,5114,Fullmetal Alchemist Brotherhood,0.517180
2,28977,Gintama,0.046209
3,9253,Steins Gate,0.555095
4,9969,Gintama 039,0.094787
...,...,...,...
9275,4310,Joshidai Ecchi Soudanshitsu,0.000000
9276,5569,Tsui no Sora,0.000000
9277,17823,Ikenai Boy,0.000592
9278,18197,Nudl Nude,0.000000


In [44]:
columns =  ['id', 'name', '1']

df1 = pd.DataFrame(c1)
df1 = pd.merge(anime, df1, on=['name','name'])
df1 = df1.drop(["genre", "type", "episodes", "rating", "members"], axis=1)
df1.columns = columns
df1

,id,name,1
0,32281,Kimi no Na wa,0.104212
1,5114,Fullmetal Alchemist Brotherhood,0.659906
2,28977,Gintama,0.074259
3,9253,Steins Gate,0.612168
4,9969,Gintama 039,0.122621
...,...,...,...
9275,4310,Joshidai Ecchi Soudanshitsu,0.000000
9276,5569,Tsui no Sora,0.000000
9277,17823,Ikenai Boy,0.000000
9278,18197,Nudl Nude,0.000000


In [45]:
columns =  ['id', 'name', '2']

df2 = pd.DataFrame(c2)
df2 = pd.merge(anime, df2, on=['name','name'])
df2 = df2.drop(["genre", "type", "episodes", "rating", "members"], axis=1)
df2.columns = columns
df2

,id,name,2
0,32281,Kimi no Na wa,0.017139
1,5114,Fullmetal Alchemist Brotherhood,0.512411
2,28977,Gintama,0.026005
3,9253,Steins Gate,0.355201
4,9969,Gintama 039,0.118794
...,...,...,...
9275,4310,Joshidai Ecchi Soudanshitsu,0.001773
9276,5569,Tsui no Sora,0.001182
9277,17823,Ikenai Boy,0.000000
9278,18197,Nudl Nude,0.000000


In [46]:
columns =  ['id', 'name', '3']

df3 = pd.DataFrame(c3)
df3 = pd.merge(anime, df3, on=['name','name'])
df3 = df3.drop(["genre", "type", "episodes", "rating", "members"], axis=1)
df3.columns = columns
df3

,id,name,3
0,32281,Kimi no Na wa,0.008051
1,5114,Fullmetal Alchemist Brotherhood,0.214611
2,28977,Gintama,0.001416
3,9253,Steins Gate,0.138651
4,9969,Gintama 039,0.014014
...,...,...,...
9275,4310,Joshidai Ecchi Soudanshitsu,0.000075
9276,5569,Tsui no Sora,0.000000
9277,17823,Ikenai Boy,0.000000
9278,18197,Nudl Nude,0.000075


In [47]:
new_df = pd.merge(df0, df1, on=['name','name'])
new_df = pd.merge(new_df, df2, on=['name','name'])
new_df = pd.merge(new_df, df3, on=['name','name'])
new_df = pd.merge(new_df, anime, on=['name', 'name'])
new_df = new_df.drop(["genre", "type", "episodes", "rating", "members", "id_x", "id_y"], axis=1)
new_df

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: Passing 'suffixes' which cause duplicate columns {'id_x'} in the result is deprecated and will raise a MergeError in a future version.
  This is separate from the ipykernel package so we can avoid doing imports until


,name,0,1,2,3,anime_id
0,Kimi no Na wa,0.079384,0.104212,0.017139,0.008051,32281
1,Fullmetal Alchemist Brotherhood,0.517180,0.659906,0.512411,0.214611,5114
2,Gintama,0.046209,0.074259,0.026005,0.001416,28977
3,Steins Gate,0.555095,0.612168,0.355201,0.138651,9253
4,Gintama 039,0.094787,0.122621,0.118794,0.014014,9969
...,...,...,...,...,...,...
9935,Joshidai Ecchi Soudanshitsu,0.000000,0.000000,0.001773,0.000075,4310
9936,Tsui no Sora,0.000000,0.000000,0.001182,0.000000,5569
9937,Ikenai Boy,0.000592,0.000000,0.000000,0.000000,17823
9938,Nudl Nude,0.000000,0.000000,0.000000,0.000075,18197


In [48]:
cols = new_df.columns.tolist()
cols = cols[-1:] + cols[:-1]
cols

['anime_id', 'name', '0', '1', '2', '3']

In [49]:
new_df = new_df[cols]
new_df

,anime_id,name,0,1,2,3
0,32281,Kimi no Na wa,0.079384,0.104212,0.017139,0.008051
1,5114,Fullmetal Alchemist Brotherhood,0.517180,0.659906,0.512411,0.214611
2,28977,Gintama,0.046209,0.074259,0.026005,0.001416
3,9253,Steins Gate,0.555095,0.612168,0.355201,0.138651
4,9969,Gintama 039,0.094787,0.122621,0.118794,0.014014
...,...,...,...,...,...,...
9935,4310,Joshidai Ecchi Soudanshitsu,0.000000,0.000000,0.001773,0.000075
9936,5569,Tsui no Sora,0.000000,0.000000,0.001182,0.000000
9937,17823,Ikenai Boy,0.000592,0.000000,0.000000,0.000000
9938,18197,Nudl Nude,0.000000,0.000000,0.000000,0.000075


In [50]:
new_df.to_csv('clusters.csv', index=False)